# 07 — Intro to Streamlit

In this notebook you’ll learn what **Streamlit** is and how to build a **minimal UI** for AI apps.

We’ll skip the standalone demo and go straight to connecting Streamlit with a real **FastAPI** backend — the cleanest pattern for production-ready prototypes.

## 🧠 What is Streamlit?

**Streamlit** is a Python framework for building **simple web apps** with minimal code. It’s ideal for:
- Rapid **prototyping** of AI demos (chat, RAG, agents)
- Internal tools and dashboards
- Sharing experiments with non-developers

You write **pure Python**, and Streamlit turns it into a reactive web UI — with widgets like inputs, buttons, and markdown displays — no HTML or JavaScript required.

## ⚡ Why Streamlit for AI apps?

- ⚡ **Fast iteration**: build a working UI in minutes.
- 🧱 **Zero JS**: write everything in Python, Streamlit handles the frontend.
- 🧪 **Perfect for demos**: quick interfaces for LangChain, LangGraph, or CrewAI apps.
- ☁️ **Easy deployment**: run locally or deploy on Streamlit Cloud, Hugging Face, or similar services.

It’s especially popular for **AI prototypes** — combining simplicity, readability, and real-time interactivity.

## 🧩 Basic idea

A Streamlit app is simply a **Python script** you run with:

```bash
streamlit run app_streamlit.py
```

You compose UI elements using functions like:

- `st.title("My App")`
- `st.text_input("Your question")`
- `st.button("Submit")`

Each time the user interacts, Streamlit automatically re-runs your script to refresh the interface.

Now, let’s connect Streamlit directly to your **FastAPI** backend to turn this into a live AI chat UI.

## 🌐 Connecting Streamlit to a FastAPI backend

This approach makes Streamlit the **frontend** and FastAPI the **backend**. 
Streamlit handles the input/output and sends user messages to the FastAPI `/chat` endpoint, which runs your LLM logic (OpenAI, Groq, etc.).

In [ ]:
# app_streamlit.py — Streamlit UI calling a FastAPI /chat endpoint
import os
import requests
import streamlit as st

# --- Page setup ---
st.set_page_config(page_title="Streamlit + FastAPI", page_icon="🛰️")
st.title("Streamlit UI → FastAPI /chat")

# --- Config ---
# Read API URL from environment or use default localhost
DEFAULT_API_URL = "http://localhost:8000/chat"
API_URL = os.getenv("CHAT_API_URL", DEFAULT_API_URL)

# --- Session state ---
if "history" not in st.session_state:
    st.session_state.history = []  # [(role, text)]
if "thread_id" not in st.session_state:
    st.session_state.thread_id = "demo-user-1"

# --- Input UI ---
user_msg = st.text_input("Your message", key="msg_input_api")
send = st.button("Send to API")

# --- Helper: call FastAPI /chat endpoint ---
def call_api(message: str, thread_id: str | None = None) -> str:
    """Send POST request to FastAPI backend and return reply text."""
    payload = {"message": message}
    if thread_id:
        payload["thread_id"] = thread_id
    try:
        r = requests.post(API_URL, json=payload, timeout=30)
        r.raise_for_status()
        data = r.json()
        return data.get("reply", "(no reply field)")
    except Exception as e:
        return f"Error calling API: {e}"

# --- Handle Send button ---
if send and user_msg.strip():
    st.session_state.history.append(("user", user_msg))
    with st.spinner("Contacting FastAPI backend..."):
        reply = call_api(user_msg, st.session_state.thread_id)
    st.session_state.history.append(("assistant", reply))
    # Trigger rerun for real-time update (compatible with old/new Streamlit)
    if hasattr(st, "rerun"):
        st.rerun()
    elif hasattr(st, "experimental_rerun"):
        st.experimental_rerun()

# --- Display conversation ---
st.subheader("Conversation")
for role, text in st.session_state.history:
    st.markdown(f"**{role.title()}:** {text}")

# --- Config panel ---
with st.expander("Config"):
    st.write("API_URL:", API_URL)
    st.write("thread_id:", st.session_state.thread_id)

st.caption("Lightweight Streamlit UI. All LLM logic handled by FastAPI backend.")

## ▶️ Run locally

Start your **FastAPI backend** first:
```bash
uv run uvicorn main:app --reload --port 8000
```

Then, run the Streamlit UI:
```bash
uv run streamlit run app_streamlit.py
```

Visit [http://localhost:8501](http://localhost:8501) to chat with your model through the FastAPI endpoint.

## ⚙️ Troubleshooting

- **Connection error:** Make sure FastAPI is running on port 8000.
- **(no reply field):** Your backend didn’t return a `{"reply": ...}` response.
- **Remote API:** To use a hosted backend, set an environment variable before launching Streamlit:
```bash
export CHAT_API_URL="https://myserver.com/chat"
uv run streamlit run app_streamlit.py
```

## ✅ Summary

- **Streamlit** builds fast, interactive UIs entirely in Python.
- It pairs perfectly with **FastAPI** for a clean frontend-backend separation.
- This setup scales naturally — plug in LangGraph, CrewAI, or RAG systems behind `/chat`.
- Keep Streamlit lightweight: just inputs, history, and visualization.

🚀 Next, you’ll integrate **LangSmith monitoring** to track and debug your AI app end-to-end.